In [72]:
from bs4 import BeautifulSoup as bsoup #need bs for that sweet, sweet web scraping
from urllib.request import urlopen #need to get url
from datetime import date #need to pull the current date
import calendar #calendar can find the name of the month
import markovify

#find today's date and fetch the wiki page
today = date.today()
url = "https://en.wikipedia.org/wiki/"+calendar.month_name[today.month]+"_"+str(today.day)
print(url) #just checking

#open the page and turn it into soup
content = urlopen(url).read()
soup = bsoup(content, "lxml")
#print(soup.prettify())

https://en.wikipedia.org/wiki/June_10


In [80]:
#get
#events = soup.find_all("ul")[1]
events_li = soup.find_all("ul")[1].find_all("li")
#print(events)

AttributeError: ResultSet object has no attribute 'get_text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [85]:
#lists for the event dates and the events themselves
dates = []
events = []
comp = []

#iterate through all events in the list
for event in events_li:
    comp.append(event.get_text())
    #split them to separate date and description
    temp = event.get_text().split("–")
    #add date to dates
    dates.append(temp[0])
    #take a slice of the rest and join it to take care of the '–' char elsewhere
    events.append("".join(temp[1::]))
# check it out
# print(dates)
# for each in events:
#    print(each)

In [89]:
#time to get into real markov stuff
text_model = markovify.Text("".join(events))
complete_model = markovify.Text("".join(comp))

#this prints out sentences for the split up events
# print("SHORT SENTENCE")
# for i in range(10):
#     print(text_model.make_short_sentence(140))
# print("\nLONG SENTENCE")
# for j in range(10):
#     print(text_model.make_sentence())

#testing with full data
for k in range(10):
    print(comp)
#     print(complete_model.make_short_sentence(140))

['671 – Emperor Tenji of Japan introduces a water clock (clepsydra) called Rokoku. The instrument, which measures time and indicates hours, is placed in the capital of Ōtsu.', '1190 – Third Crusade: Frederick I Barbarossa drowns in the river Saleph while leading an army to Jerusalem.', '1329 – The Battle of Pelekanon results in a Byzantine defeat by the Ottoman Empire.', "1523 – Copenhagen is surrounded by the army of Frederick I of Denmark, as the city won't recognise him as the successor of Christian II of Denmark.", '1539 – Council of Trent: Pope Paul III sends out letters to his bishops, delaying the Council due to war and the difficulty bishops had traveling to Venice.', '1596 – Willem Barents and Jacob van Heemskerk discover Bear Island.', "1619 – Thirty Years' War: Battle of Záblatí, a turning point in the Bohemian Revolt.", '1624 – Signing of the Treaty of Compiègne between France and the Netherlands.', '1692 – Salem witch trials: Bridget Bishop is hanged at Gallows Hill near S